In [2]:
import ehr_utils, ehr_models
import os
import pickle
import numpy as np
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.utils import compute_sample_weight
from imblearn.over_sampling import SMOTE
import shutil


In [3]:
print("=== 步骤1：获取训练集和测试集 ===")
X_train, X_test, y_train, y_test = ehr_utils.preprocess_ehr_train_test_data('data_processed/benbu_baseline_cleaned_onehot.csv')
print(f"原始训练集大小：{X_train.shape}")
print(f"原始测试集大小：{X_test.shape}")

print("\n=== 步骤2：准备所有模型 ===")
# 定义所有要训练的模型
all_models = [
    'AdaBoost', 'DecisionTree', 'GaussianNB', 'GradientBoosting', 'LightGBM', 'LinearDiscriminantAnalysis', 'LogisticRegression', 'MLPClassifier', 'RandomForest', 'XGBoost'
]
print(f"将训练以下 {len(all_models)} 个模型：")
for i, model_name in enumerate(all_models, 1):
    print(f"  {i}. {model_name}")

# 存储所有模型的结果
all_models_results = {}
total_start_time = time.time()
print("\n" + "=" * 80)
print("开始训练所有模型...")
print("-" * 80)
shutil.rmtree('models', ignore_errors=True)
os.makedirs('models', exist_ok=True)

n_splits = 5

for model_idx, model_name in enumerate(all_models, 1):
    print(f"\n{'='*60}")
    print(f"正在训练模型 {model_idx}/{len(all_models)}：{model_name}")
    print(f"{'='*60}")
    model_start_time = time.time()

    try:
        # 获取模型
        model = ehr_models.get_model(model_name)
        print(f"模型参数：{model.get_params()}")
        print(f"\n=== 开始 {model_name} 五折交叉验证 ===")
        # 交叉验证设置
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        cv_results = {'train_aucs': [], 'val_aucs': [], 'models': [], 'fold_results': []}

        # K折交叉验证
        for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
            print(f'\n----- Fold {fold + 1}/{n_splits} -----')
            X_fold_train = X_train.iloc[train_idx]
            X_fold_val = X_train.iloc[val_idx]
            y_fold_train = y_train.iloc[train_idx].values  
            y_fold_val = y_train.iloc[val_idx].values  

            # 计算样本权重
            sample_weights = compute_sample_weight(class_weight='balanced', y=y_fold_train)
            pos_mask = (y_fold_train == 1)
            sample_weights[pos_mask] *= 1.1

            # 每次都重新获取模型
            fold_model = ehr_models.get_model(model_name)

            if model_name == 'LinearDiscriminantAnalysis':
                # LDA 不支持样本权重，使用 SMOTE
                smote = SMOTE(random_state=42)
                X_train_fold_res, y_train_fold_res = smote.fit_resample(X_fold_train, y_fold_train)  # type: ignore
                fold_model.fit(X_train_fold_res, y_train_fold_res)  # type: ignore
            else:
                fold_model.fit(X_fold_train, y_fold_train, sample_weight=sample_weights)  # type: ignore

            # 预测
            y_train_proba = np.asarray(fold_model.predict_proba(X_fold_train))[:, 1]
            y_val_proba = np.asarray(fold_model.predict_proba(X_fold_val))[:, 1]

            # 计算 AUC
            train_auc = roc_auc_score(y_fold_train, y_train_proba)
            val_auc = roc_auc_score(y_fold_val, y_val_proba)
            print(f"训练集 AUC：{train_auc:.4f}，验证集 AUC：{val_auc:.4f}，过拟合：{train_auc - val_auc:.4f}")
            # 保存结果
            cv_results['train_aucs'].append(train_auc)
            cv_results['val_aucs'].append(val_auc)
            cv_results['models'].append(fold_model)
            cv_results['fold_results'].append({'fold': fold, 'train_auc': train_auc, 'val_auc': val_auc, 'train_idx': train_idx, 'val_idx': val_idx})

        # 计算平均结果
        mean_train_auc = np.mean(cv_results['train_aucs'])
        mean_val_auc = np.mean(cv_results['val_aucs'])
        std_train_auc = np.std(cv_results['train_aucs'])
        std_val_auc = np.std(cv_results['val_aucs'])

        print(f"\n{model_name} 五折交叉验证结果汇总：")
        print(f"各折验证集 AUC: {[f'{auc:.4f}' for auc in cv_results['val_aucs']]}")
        print(f"平均训练集 AUC: {mean_train_auc:.4f} ± {std_train_auc:.4f}")
        print(f"平均验证集 AUC: {mean_val_auc:.4f} ± {std_val_auc:.4f}")
        print(f"平均过拟合程度：{mean_train_auc - mean_val_auc:.4f}")
        # 选择最佳模型（验证集 AUC 最高的）
        best_fold_idx = np.argmax(cv_results['val_aucs'])
        best_model = cv_results['models'][best_fold_idx]
        best_val_auc = cv_results['val_aucs'][best_fold_idx]
        print(f"最佳模型来自第 {best_fold_idx+1} 折，验证集 AUC：{best_val_auc:.4f}")

        # 保存最佳模型
        os.makedirs('models', exist_ok=True)
        model_filename = f'models/{model_name}_cv_{mean_val_auc:.4f}.pkl'
        with open(model_filename, 'wb') as f:
            pickle.dump(best_model, f)
        print(f"最佳模型已保存到：{model_filename}")

        # 计算训练时间
        model_end_time = time.time()
        model_duration = model_end_time - model_start_time

        # 保存模型结果
        all_models_results[model_name] = {
            'mean_train_auc': mean_train_auc,
            'mean_val_auc': mean_val_auc,
            'std_train_auc': std_train_auc,
            'std_val_auc': std_val_auc,
            'best_val_auc': best_val_auc,
            'overfitting': mean_train_auc - mean_val_auc,
            'training_time': model_duration,
            'best_model': best_model,
            'cv_results': cv_results
        }
        print(f"✓ {model_name} 训练完成！用时：{model_duration:.2f} 秒")

    except Exception as e:
        print(f"✗ {model_name} 训练失败：{str(e)}")
        all_models_results[model_name] = {'error': str(e), 'training_time': time.time() - model_start_time}

# 计算总用时
total_end_time = time.time()
total_duration = total_end_time - total_start_time

print("\n" + "=" * 80)
print("所有模型训练完成！")
print("=" * 80)

=== 步骤1：获取训练集和测试集 ===
总特征数：96，需要log1p变换的特征数：66
训练集样本数：8000，验证集样本数：2000
原始训练集大小：(8000, 96)
原始测试集大小：(2000, 96)

=== 步骤2：准备所有模型 ===
将训练以下 10 个模型：
  1. AdaBoost
  2. DecisionTree
  3. GaussianNB
  4. GradientBoosting
  5. LightGBM
  6. LinearDiscriminantAnalysis
  7. LogisticRegression
  8. MLPClassifier
  9. RandomForest
  10. XGBoost

开始训练所有模型...
--------------------------------------------------------------------------------

正在训练模型 1/10：AdaBoost
模型参数：{'algorithm': 'deprecated', 'estimator': None, 'learning_rate': 0.6856988052045783, 'n_estimators': 200, 'random_state': 42}

=== 开始 AdaBoost 五折交叉验证 ===

----- Fold 1/5 -----
训练集 AUC：0.5869，验证集 AUC：0.4865，过拟合：0.1004

----- Fold 2/5 -----
训练集 AUC：0.5834，验证集 AUC：0.5239，过拟合：0.0595

----- Fold 3/5 -----
训练集 AUC：0.5440，验证集 AUC：0.5083，过拟合：0.0358

----- Fold 4/5 -----
训练集 AUC：0.5954，验证集 AUC：0.5015，过拟合：0.0939

----- Fold 5/5 -----
训练集 AUC：0.5037，验证集 AUC：0.5010，过拟合：0.0026

AdaBoost 五折交叉验证结果汇总：
各折验证集 AUC: ['0.4865', '0.5239', '0.5083', '0.5015', '0.

d:\Cache\Conda\envs\EHR172\Lib\site-packages\sklearn\linear_model\_logistic.py:1305: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(


训练集 AUC：0.5662，验证集 AUC：0.4960，过拟合：0.0703

----- Fold 2/5 -----


d:\Cache\Conda\envs\EHR172\Lib\site-packages\sklearn\linear_model\_logistic.py:1305: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(


训练集 AUC：0.5702，验证集 AUC：0.4914，过拟合：0.0788

----- Fold 3/5 -----


...

训练集 AUC：0.5740，验证集 AUC：0.4882，过拟合：0.0858

----- Fold 5/5 -----


d:\Cache\Conda\envs\EHR172\Lib\site-packages\sklearn\linear_model\_logistic.py:1305: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(


训练集 AUC：0.5640，验证集 AUC：0.5132，过拟合：0.0507

LogisticRegression 五折交叉验证结果汇总：
各折验证集 AUC: ['0.4960', '0.4914', '0.4653', '0.4882', '0.5132']
平均训练集 AUC: 0.5699 ± 0.0043
平均验证集 AUC: 0.4908 ± 0.0154
平均过拟合程度：0.0791
最佳模型来自第 5 折，验证集 AUC：0.5132
最佳模型已保存到：models/LogisticRegression_cv_0.4908.pkl
✓ LogisticRegression 训练完成！用时：1.27 秒

正在训练模型 8/10：MLPClassifier
模型参数：{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': True, 'epsilon': 1e-08, 'hidden_layer_sizes': [200, 50], 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 1000, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 42, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}

=== 开始 MLPClassifier 五折交叉验证 ===

----- Fold 1/5 -----
训练集 AUC：0.5271，验证集 AUC：0.5164，过拟合：0.0106

----- Fold 2/5 -----
训练集 AUC：0.5379，验证集 AUC：0.4966，过拟合：0.0413

----- F

d:\Cache\Conda\envs\EHR172\Lib\site-packages\xgboost\core.py:729: UserWarning: [06:04:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


训练集 AUC：0.8396，验证集 AUC：0.4910，过拟合：0.3486

----- Fold 2/5 -----
训练集 AUC：0.8371，验证集 AUC：0.5022，过拟合：0.3349

----- Fold 3/5 -----
训练集 AUC：0.8354，验证集 AUC：0.4814，过拟合：0.3539

----- Fold 4/5 -----
训练集 AUC：0.8279，验证集 AUC：0.4974，过拟合：0.3305

----- Fold 5/5 -----
训练集 AUC：0.8383，验证集 AUC：0.5137，过拟合：0.3246

XGBoost 五折交叉验证结果汇总：
各折验证集 AUC: ['0.4910', '0.5022', '0.4814', '0.4974', '0.5137']
平均训练集 AUC: 0.8357 ± 0.0041
平均验证集 AUC: 0.4972 ± 0.0108
平均过拟合程度：0.3385
最佳模型来自第 5 折，验证集 AUC：0.5137
最佳模型已保存到：models/XGBoost_cv_0.4972.pkl
✓ XGBoost 训练完成！用时：6.45 秒

所有模型训练完成！
